In [32]:
import pandas as pd
import numpy as np
import pyreadr
from scipy import stats
import statsmodels.api as sm

In [33]:
file_path = "/Users/yaolangzhong/Nottingham_Replication/original_code/"

In [34]:
data = pyreadr.read_r(file_path + "data/straddle_returns_fomc.RData")['data']
# Convert the 'date' column to datetime type and create a new column 'ym'
data['date'] = pd.to_datetime(data['date'])
data['exp'] = pd.to_datetime(data['exp'])
data['ym'] = data['date'].dt.year*100 + data['date'].dt.month
# Filter data
data = data[(data['date'].dt.year >= 1994) &
            ((data['ym'] <= 200706) | (data['ym'] >= 200907))]


In [43]:
def return_stats(R):
    skew = stats.skew(R)
    kurt = stats.kurtosis(R)
    mod = sm.OLS(R, np.ones(len(R))).fit()
    tstat = mod.tvalues[0]
    res = pd.Series([f"{R.mean():4.1f}",
                     round(R.median(), 1),
                     round(R.std(), 1),
                     round(skew, 1),
                     round(kurt, 1),
                     round(tstat, 1),
                     round(abs(np.sqrt(8) * R.mean()) / R.std(), 1),
                     len(R)])
    res.index = ["Mean", "Median", "SD", "Skewness", "Kurtosis", "t-statistic", "Sharpe ratio", "Observations"]
    return res


In [44]:
# Define empty dataframes
tbl = pd.DataFrame(np.nan, index=range(8), columns=range(6))
tbl.columns = [f"ED{j}" for j in range(1, 7)]
tbl.index = ["Mean", "Median", "SD", "Skewness", "Kurtosis", "t-statistic", "Sharpe ratio", "Observations"]

tbl2 = tbl.copy()

# Loop through the columns
for j in range(1, 7):
    tmp = data[data['j']==j]
    # Relative returns
    tbl[f"ED{j}"] = return_stats(tmp['Rrel'].dropna())
    # Absolute returns
    tbl2[f"ED{j}"] = return_stats(tmp['Rabs'].dropna())

print(tbl)
print(tbl2)


               ED1   ED2   ED3   ED4   ED5   ED6
Mean          -9.1  -4.5  -3.0  -2.3  -2.1  -1.6
Median        -8.3  -4.2  -2.8  -2.2  -1.7  -1.4
SD            11.9   6.1   4.5   4.1   3.7   3.3
Skewness      -0.3  -0.6   0.1  -0.5  -3.2  -0.9
Kurtosis       2.6   4.6   4.5  16.6  20.7  11.9
t-statistic  -10.8 -10.5  -9.1  -7.8  -8.0  -6.8
Sharpe ratio   2.2   2.1   1.8   1.6   1.6   1.4
Observations   197   197   197   197   197   194
               ED1   ED2   ED3   ED4   ED5   ED6
Mean          -1.3  -1.3  -1.4  -1.4  -1.4  -1.3
Median        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
SD             1.8   1.9   2.2   2.2   2.4   2.4
Skewness      -0.8  -0.7  -0.6  -1.1  -1.4  -0.8
Kurtosis       3.3   3.8   2.7   3.7   5.1   3.8
t-statistic  -10.2 -10.0  -8.6  -8.7  -8.4  -7.9
Sharpe ratio   2.1   2.0   1.7   1.7   1.7   1.6
Observations   197   197   197   197   197   194


In [45]:
with open("output/table_C3.tex", 'w') as f:
    f.write("& ED1 & ED2 & ED3 & ED4 & ED5 & ED6 \\\\ \n")
    f.write("\\midrule \n")
    f.write("\\multicolumn{6}{l}{\\emph{Relative returns}}\\\\ \n")
    f.write(tbl.iloc[0:7].to_latex(header=False, column_format='r' * 7))
    f.write("\\midrule \n")
    f.write("\\multicolumn{6}{l}{\\emph{Absolute returns}}\\\\ \n")
    f.write(tbl2.iloc[0:7].to_latex(header=False, column_format='r' * 7))
    f.write("\\midrule \n")
    f.write("Observations " + " ".join([f"& {x}" for x in tbl.iloc[7]]) + " \\\\ \n")
